In [1]:
%cd ../src
%load_ext autoreload
%autoreload 2

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

In [9]:
from dataloaders.shapenet.shapenet_loader import ShapeNet

categories = ['airplane']
path = "../data/ShapeNet"

te = ShapeNet(path, "test", 2048, categories, load_renders=True)

Loading (test) renders for airplane (02691156):   0%|          | 0/808 [00:00<?, ?it/s]

In [10]:
from utils.helper_functions import normalize_to_unit_sphere
from metrics.chamfer_dist import ChamferDistanceL2
from metrics.PyTorchEMD.emd import earth_mover_distance as EMD

cd = ChamferDistanceL2()
point_clouds = normalize_to_unit_sphere(torch.stack([te[i]['pc'] for i in range(len(te))])).cuda()
P = torch.randn(2048, 3, requires_grad=True, device='cuda')

optim = torch.optim.Adam([P], lr=0.05)

for _ in range(150):
    loss = sum(cd(P.unsqueeze(0), pc.unsqueeze(0)) for pc in point_clouds) / len(point_clouds)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if _ % 10 == 0:
        print(f"Iteration {_}, Loss: {loss.item()}")
print("Optimized Point Cloud:")

Iteration 0, Loss: 1.6942265033721924
Iteration 10, Loss: 0.6120258569717407
Iteration 20, Loss: 0.2140909731388092
Iteration 30, Loss: 0.07753150910139084
Iteration 40, Loss: 0.03064647503197193
Iteration 50, Loss: 0.014449182897806168
Iteration 60, Loss: 0.008860098198056221
Iteration 70, Loss: 0.006755447015166283
Iteration 80, Loss: 0.00593292061239481
Iteration 90, Loss: 0.005550343077629805
Iteration 100, Loss: 0.005357465241104364
Iteration 110, Loss: 0.0052574980072677135
Iteration 120, Loss: 0.005203566048294306
Iteration 130, Loss: 0.005169901996850967
Iteration 140, Loss: 0.005148820113390684
Optimized Point Cloud:


In [12]:
from utils.visualization import display_pointclouds_grid
import numpy as np
P = 5 * P
display_pointclouds_grid(np.array([P.detach().cpu()]), point_size=0.3)

Output()